## migracia

* Alica Kvasnakova
* Simona Makarova
* Katarina Orsagova
* Kristina Knappova

### data frame preparation

Data have been cleaned and prepared for use. We merged another tables (adding goe data) into our original table.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.float_format = '{:,.2f}'.format

import geopandas as gpd
import geoplot as gplt

In [3]:
url = "estat_migr_pop1ctz$defaultview_filtered.tsv"

data = pd.read_csv(url, sep='\t')

#fixing names for columns
first_col_name = data.columns[0]  

split_cols = data[first_col_name].str.split(',', expand=True)
split_cols.columns = ['freq', 'citizen', 'age', 'unit', 'sex', 'alpha-2']  

data = pd.concat([split_cols, data.drop(columns=[first_col_name])], axis=1)

names = [i.strip() for i in data.columns.values]
data.columns.values[:] = names

print(data.columns)

#fixing missing and approximate data
data = data.replace(': ',np.nan)

data = data.map(lambda x: x if len(str(x).split())==1 else str(x).split()[0])
# data = data.apply(pd.to_numeric, errors='ignore')
data = data.map(lambda x: int(x) if str(x).isdigit() else x)

#adding ISO3 codes for countries
link = 'country_codes.csv'
country_codes = pd.read_csv(link)
country_codes = country_codes[['alpha-2','alpha-3','name']]
country_codes['name'] = country_codes['name'].replace('Netherlands, Kingdom of the', 'Netherlands')
country_codes.rename(columns={'alpha-3': 'ISO3'}, inplace=True)
data = data.merge(country_codes,on='alpha-2')

#adding info for country shapes
country_shape = gpd.read_file("https://bbrejova.github.io/viz/data/country_boundaries.geojson")
country_shape = country_shape[country_shape['Name'] != 'N. Cyprus']
country_shape = country_shape[['ISO3','geometry']]
data = data.merge(country_shape,on='ISO3')


display(data.head(100))
print(data.shape)

data.to_csv("rozdelena_tabulka.csv", index=False)

Index(['freq', 'citizen', 'age', 'unit', 'sex', 'alpha-2', '2015', '2016',
       '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024'],
      dtype='object')


,freq,citizen,age,unit,sex,alpha-2,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,ISO3,name,geometry
0,A,EU28_FOR,TOTAL,NR,F,AT,288028,310759,329981,348012,365125,383295,NaN,NaN,NaN,NaN,AUT,Austria,"POLYGON ((16.97967 48.1235, 16.90375 47.71487,..."
1,A,EU28_FOR,TOTAL,NR,F,BE,416232,428008,434889,440355,448655,457258,NaN,NaN,NaN,NaN,BEL,Belgium,"POLYGON ((6.15666 50.80372, 6.04307 50.12805, ..."
2,A,EU28_FOR,TOTAL,NR,F,CH,594052,610566,622893,630371,637330,645916,NaN,NaN,NaN,NaN,CHE,Switzerland,"POLYGON ((9.59423 47.52506, 9.63293 47.3476, 9..."
3,A,EU28_FOR,TOTAL,NR,F,CY,55730,57716,57673,59787,63044,62935,NaN,NaN,NaN,NaN,CYP,Cyprus,"POLYGON ((32.73178 35.14003, 32.91957 35.08783..."
4,A,EU28_FOR,TOTAL,NR,F,CZ,72465,76736,81670,86132,91296,96186,NaN,NaN,NaN,NaN,CZE,Czechia,"POLYGON ((15.017 51.10667, 15.49097 50.78473, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,A,NAT,TOTAL,NR,F,IE,2096566,2113540,2140010,2158148,2187792,2209973,2233904,2274553,2280509,2289258,IRL,Ireland,"POLYGON ((-6.19788 53.86757, -6.03299 53.15316..."
96,A,NAT,TOTAL,NR,F,IS,152218,152777,153732,154921,155871,156863,158091,159907,160376,159434,ISL,Iceland,"POLYGON ((-14.5087 66.45589, -14.73964 65.8087..."
97,A,NAT,TOTAL,NR,F,IT,28502891,28416843,28353665,28236011,28103605,27983433,27722737,27648663,27558567,27473494,ITA,Italy,"MULTIPOLYGON (((10.4427 46.89355, 11.04856 46...."
98,A,NAT,TOTAL,NR,F,LT,1570481,1554534,1535286,1517240,1507800,1501028,1496615,1488535,1480495,1474354,LTU,Lithuania,"POLYGON ((26.49433 55.61511, 26.58828 55.16718..."


(525, 19)
